In [64]:
def find_positions(A, B):
    positions = []
    added_elements = set()  
    for num in A:
        if num not in added_elements and num in B:
            positions.append(B.index(num))
            added_elements.add(num)
    return positions, list(added_elements)
    
def extract_elements_by_positions(remain_map, image_id, pred):
    ind = []
    ind_temp = []
    ind_multiple = []
    

    for p in pred:

        count = image_id.count(p)
        
        if count > 1:
            ind_multiple.append(p)
            continue

        index = image_id.index(p)
        ind_temp.append(index)

    for p in ind_multiple:
        min_distance_sum = float('inf')
        min_index = None
        

        for index, value in enumerate(image_id):
            if value == p:

                distance_sum = sum(abs(index - temp_index) for temp_index in ind_temp)
                

                if distance_sum < min_distance_sum and index not in ind_temp:
                    min_distance_sum = distance_sum
                    min_index = index
        

        if min_index is not None:
            ind_temp.append(min_index)
    

    for index in ind_temp:
        ind.append(remain_map[index])
    
    return ind


def filter_elements(J, K):
    filtered_list = [elem for elem in J if elem in K]
    return filtered_list


In [63]:
import json
data = [json.loads(q) for q in open('your_results', "r")]

In [65]:
loc_list = []
null_count = 0
intersect = []
reli = []
prec = []
iou = []
for item in data:
    M = item['image_map']
    A = item['predict_tokens'][1:]
    A_ = filter_elements(A,M)
    B = item['gt_tokens']
    num_pred = len(A)
    corr,corr_list = find_positions(A,B)
    ratio = len(corr)/num_pred
    precision = len(corr)/len(B)
    union = len(A_)+len(B)-len(corr)
    iou_ = len(corr)/union
    iou.append(iou_)
    prec.append(precision)
    intersect.append(ratio)
    C = item['remained_map']
    D = item['image_map']
    findd = extract_elements_by_positions(C,D,A_)
    loc = {}
    loc['loc'] = findd
    loc['corr'] = corr_list
    loc['gt_box'] = item['gt_answer']
    loc['image'] = item['file_name']
    loc['object'] = item['prompt']
    loc['precision'] = precision #### intersect with gt
    loc['pred_intersect_gt'] = '{:.3f}'.format(ratio) ###previous overlap, 
    loc['iou'] = iou_
    loc['reliability'] = '{:.3f}'.format(len(A_)/len(A)) ###how many of pred is usable?
    reli.append(len(A_)/len(A))
    if ratio==0:
        null_count = null_count + 1
    loc_list.append(loc)

In [66]:
import numpy as np
reli = np.array(reli)
intersect = np.array(intersect)
prec = np.array(prec)
iou = np.array(iou)
print(reli.mean())
print(intersect.mean())
print(prec.mean())
print(iou.mean())

0.4957919046483946
0.334850178651876
0.2676380397075303
0.21899404960448327


In [62]:
with open('output_file', 'w', encoding='utf-8') as f:
    json.dump(loc_list, f, ensure_ascii=False, indent=4)